In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250217_GreenBlue_Geometry_Data/RunData'

In [ ]:
df = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
df["FlyID"].nunique()


In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_displacement(df[df['Scene']=='Choice_noBG'], min_disp=0, max_disp=500)

In [ ]:
plot_trajectories(df_normal, 'normal')

In [ ]:
import pandas as pd
import numpy as np

# 1. Define your different config groups and their corresponding goals
#    (You can rename these lists and add/remove as needed.)
configs_2_goals_front = [
    "BinaryChoice_constantSize_GreenCylinder_GreenCylinder.json",
    "BinaryChoice11_constantSize_BlackCylinder_BlackCylinder.json",
    "BinaryChoice_constantSize_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_GreenCylinder.json"
]

# For these configs, the left and right goals are at:
# (-10.416, 59.088) and (10.416, 59.088).
# Adjust threshold or other parameters as needed.
left_goal_1 = (-10.416, 59.088)
right_goal_1 = (10.416, 59.088)

# If you have other configs that use 2 goals with a different positioning:
configs_2_goals_side = [
    "BinaryChoice_constantSize_180deg_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_180deg_BlueCylinder_GreenCylinder.json"
]
left_goal_2 = (-60, 0)
right_goal_2 = (60, 0)

configs_2_goals_diag = [
    "BinaryChoice_constantSize_90deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_GreenCylinder_BlueCylinder.json"
]
left_goal_3 = (-42.43, 42.43)
right_goal_3 = (42.43, 42.43)

# For 3-goal configs:
configs_3_goals = [
    "3Cylinders111_constantSize.json"
]
triple_center_goal = (0, 60)
triple_left_goal = (-20.5212, 56.381557)
triple_right_goal = (20.5212, 56.381557)

# For single-goal configs (if needed):
configs_center_only = [
]
center_goal = (0, 60)


# 2. Subset your df_normal by each group and call get_first_goal_reached
def get_first_goal_reached(
    df_normal,
    goals,            # A list of (goal_name, (x, z)) tuples
    threshold=3.5
):
    """
    Given a dataframe of trial data, determine the first goal reached 
    and the time at which it was reached for each UniqueTrialID.
    
    Parameters
    ----------
    df_normal : pd.DataFrame
        The dataframe containing trial data. Must contain columns:
        ['UniqueTrialID', 'ConfigFile', 'trial_time', 'GameObjectPosX', 'GameObjectPosZ'].
    goals : list of (str, (float, float))
        A list of (goal_name, (x, z)) to check.
    threshold : float, optional
        Distance threshold below which a goal is considered reached.

    Returns
    -------
    pd.DataFrame
        A dataframe with one row per UniqueTrialID, including:
        ['UniqueTrialID', 'ConfigFile', 'FirstReachedGoal', 'GoalReachedTime'].
    """
    
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    results = []
    
    # Group by UniqueTrialID
    for trial_id, trial_data in df_normal.groupby('UniqueTrialID'):
        config = trial_data['ConfigFile'].iloc[0]
        
        # Sort by time
        trial_data = trial_data.sort_values(by='trial_time')
        
        first_reached = None
        reached_time = None
        
        for idx, row in trial_data.iterrows():
            participant_pos = (row['GameObjectPosX'], row['GameObjectPosZ'])
            
            # Check each goal
            for goal_name, goal_pos in goals:
                if distance(participant_pos, goal_pos) <= threshold:
                    first_reached = goal_name
                    reached_time = row['trial_time']
                    break
            
            if first_reached is not None:
                break
        
        results.append((trial_id, config, first_reached, reached_time))
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results, columns=[
        'UniqueTrialID',
        'ConfigFile',
        'FirstReachedGoal',
        'GoalReachedTime'
    ])
    
    return results_df


# 3. Filter and run for each group, then concatenate

# --- Group 1: 2 goals (front) ---
df_2_goals_front = df_normal[df_normal['ConfigFile'].isin(configs_2_goals_front)].copy()
results_2_goals_front = get_first_goal_reached(
    df_2_goals_front,
    goals=[
        ('left', left_goal_1),
        ('right', right_goal_1)
    ],
    threshold=3.5
)

# --- Group 2: 2 goals (left=-60,0 ; right=60,0) ---
df_2_goals_side = df_normal[df_normal['ConfigFile'].isin(configs_2_goals_side)].copy()
results_2_goals_side = get_first_goal_reached(
    df_2_goals_side,
    goals=[
        ('left', left_goal_2),
        ('right', right_goal_2)
    ],
    threshold=3.5
)

# --- Group 3: 2 goals (diagonal) ---
df_2_goals_diag = df_normal[df_normal['ConfigFile'].isin(configs_2_goals_diag)].copy()
results_2_goals_diag = get_first_goal_reached(
    df_2_goals_diag,
    goals=[
        ('left', left_goal_3),
        ('right', right_goal_3)
    ],
    threshold=3.5
)

# --- Group 4: 3 goals ---
df_3_goals = df_normal[df_normal['ConfigFile'].isin(configs_3_goals)].copy()
results_3_goals = get_first_goal_reached(
    df_3_goals,
    goals=[
        ('triple_center', triple_center_goal),
        ('triple_left', triple_left_goal),
        ('triple_right', triple_right_goal)
    ],
    threshold=3.5
)

# --- Group 5: single center goal (if needed) ---
df_center_only = df_normal[df_normal['ConfigFile'].isin(configs_center_only)].copy()
results_center_only = get_first_goal_reached(
    df_center_only,
    goals=[('center', center_goal)],
    threshold=3.5
)

# 4. Concatenate all results
all_results = pd.concat([
    results_2_goals_front,
    results_2_goals_side,
    results_2_goals_diag,
    results_3_goals,
    results_center_only
], ignore_index=True)

# all_results now contains the first reached goal for each UniqueTrialID
# from each different config group.


In [ ]:
df['ConfigFile'].unique()

In [ ]:
all_results.to_pickle("geometry_results_df.pkl")

In [ ]:
results_df = pd.read_pickle("geometry_results_df.pkl")

In [ ]:
results_df['FirstReachedGoal'].unique()

In [ ]:
# 1. Keep only trials that actually reached a goal
valid_results = results_df.dropna(subset=['FirstReachedGoal'])

# 2. Merge the cutoff times back into df
#    We merge on 'UniqueTrialID' to get each trial's GoalReachedTime.
df_merged = pd.merge(df, valid_results[['UniqueTrialID', 'GoalReachedTime', 'FirstReachedGoal']], on='UniqueTrialID', how='inner')

# 3. Filter df so that only rows with trial_time less than or equal to the goal time are kept
df_cut = df_merged[df_merged['trial_time'] <= df_merged['GoalReachedTime']]

In [ ]:
df_cut["FlyID"].nunique()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume these from previous steps:
center_only_configs = [
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json"
]

# Identify which ConfigFiles have multiple goals
# Since we know only the two listed above are single-goal configs,
# all others are multi-goal configs.
all_configs = df_cut['ConfigFile'].unique()
multi_goal_configs = [c for c in all_configs if c not in center_only_configs]

# Merge results_df to get the goal reached information into df_cut if needed
# (Not strictly necessary if we only need ratios. We can just use results_df separately.)
# But let's have a convenient DataFrame for ratio calculations.
df_joined = pd.merge(df_cut, results_df[['UniqueTrialID', 'FirstReachedGoal']], on='UniqueTrialID', how='left')

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot each trial’s trajectory
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    
    # Make axes equal
    ax.set_aspect('equal', adjustable='box')

    # Set plot titles and labels
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # If it's a multi-goal config, calculate ratio left/right
    if config in multi_goal_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        # Compute ratio (e.g., left/total and right/total)
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        # Add text box with ratio information
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    elif config in center_only_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        count = (config_results['FirstReachedGoal'] == 'center').sum()
        # Only center goal, no ratio needed
        ax.text(0.05, 0.95, f"Center goal only: {count}", transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    
    output_dir = '/Users/apaula/Downloads'
    # Remove '.json' from config filename and prepend 'trajectories_'
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories4_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(filepath, dpi=300)

    plt.tight_layout()
    plt.show()

In [ ]:
# Identify which ConfigFiles have multiple goals or triple goals
all_configs = df_cut['ConfigFile'].unique()
configs_2_goals = [
    "BinaryChoice_constantSize_GreenCylinder_GreenCylinder.json",
    "BinaryChoice11_constantSize_BlackCylinder_BlackCylinder.json",
    "BinaryChoice_constantSize_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_180deg_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_180deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_GreenCylinder_BlueCylinder.json"
]
# Merge results_df to get the goal reached information into df_cut (if needed)
df_joined = pd.merge(
    df_cut, 
    results_df[['UniqueTrialID', 'FirstReachedGoal']], 
    on='UniqueTrialID', 
    how='left'
)

output_dir = '/Users/apaula/Downloads'

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    
    # --- 1. Plot all trajectories for this config ---
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    # Basic plot settings
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # --- 2. Compute how many times each goal was reached for this config ---
    # Filter the relevant trials from results_df
    config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]

    # 2a. Single center-goal configs
    if config in center_only_configs:
        center_count = (config_results['FirstReachedGoal'] == 'center').sum()
        ratio_text = f"Center goal only: {center_count}"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2b. Triple-goal configs
    elif config in configs_3_goals:
        # Count how many reached each of the three possible goals
        left_count = (config_results['FirstReachedGoal'] == 'triple_left').sum()
        center_count = (config_results['FirstReachedGoal'] == 'triple_center').sum()
        right_count = (config_results['FirstReachedGoal'] == 'triple_right').sum()
        
        total = left_count + center_count + right_count
        if total > 0:
            left_ratio = left_count / total
            center_ratio = center_count / total
            right_ratio = right_count / total
            ratio_text = (
                f"L: {left_count} ({left_ratio:.2f}), "
                f"C: {center_count} ({center_ratio:.2f}), "
                f"R: {right_count} ({right_ratio:.2f})"
            )
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2c. Two-goal configs (assume everything else with multiple goals is two-goal)
    elif config in configs_2_goals:
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    # 2d. If you have other special cases or an 'else' fallback:
    else:
        # If you have some unclassified configs
        ratio_text = "Unclassified Config"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # --- 3. Save and show the figure ---
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories4_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=300)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Identify which ConfigFiles have multiple goals or triple goals
all_configs = df_cut['ConfigFile'].unique()
configs_2_goals = [
    "BinaryChoice_constantSize_GreenCylinder_GreenCylinder.json",
    "BinaryChoice11_constantSize_BlackCylinder_BlackCylinder.json",
    "BinaryChoice_constantSize_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_180deg_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_180deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_GreenCylinder_BlueCylinder.json"
]
# Merge results_df to get the goal reached information into df_cut (if needed)
df_joined = pd.merge(
    df_cut, 
    results_df[['UniqueTrialID', 'FirstReachedGoal']], 
    on='UniqueTrialID', 
    how='left'
)

output_dir = '/Users/apaula/Downloads'

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    
    # --- 1. Plot all trajectories for this config ---
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    # Basic plot settings
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # --- 2. Compute how many times each goal was reached for this config ---
    # Filter the relevant trials from results_df
    config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]

    # 2a. Single center-goal configs
    if config in center_only_configs:
        center_count = (config_results['FirstReachedGoal'] == 'center').sum()
        ratio_text = f"Center goal only: {center_count}"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2b. Triple-goal configs
    elif config in configs_3_goals:
        # Count how many reached each of the three possible goals
        left_count = (config_results['FirstReachedGoal'] == 'triple_left').sum()
        center_count = (config_results['FirstReachedGoal'] == 'triple_center').sum()
        right_count = (config_results['FirstReachedGoal'] == 'triple_right').sum()
        
        total = left_count + center_count + right_count
        if total > 0:
            left_ratio = left_count / total
            center_ratio = center_count / total
            right_ratio = right_count / total
            ratio_text = (
                f"L: {left_count} ({left_ratio:.2f}), "
                f"C: {center_count} ({center_ratio:.2f}), "
                f"R: {right_count} ({right_ratio:.2f})"
            )
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2c. Two-goal configs (assume everything else with multiple goals is two-goal)
    elif config in configs_2_goals:
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    # 2d. If you have other special cases or an 'else' fallback:
    else:
        # If you have some unclassified configs
        ratio_text = "Unclassified Config"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # --- 3. Save and show the figure ---
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories4_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=300)
    
    plt.tight_layout()
    plt.show()

In [ ]:
df_joined = pd.merge(df_cut, results_df[['UniqueTrialID', 'FirstReachedGoal']], on='UniqueTrialID', how='left')

import os
import matplotlib.pyplot as plt
import pandas as pd

# Specify which config you want to plot
config_of_interest = "BinaryChoice11_constantSize_15deg_10deg.json"

# Subset df_joined to only the config of interest
df_config = df_joined[df_joined["ConfigFile"] == config_of_interest].copy()

# Check if it's a multi-goal config or a center-only config
is_multi_goal = config_of_interest not in center_only_configs

# Create an output directory if needed
output_dir = '/Users/apaula/Downloads/sizebias'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Group by FlyID
for fly_id, group_fly in df_config.groupby('FlyID'):
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot each trial’s trajectory for this FlyID
    for trial_id, trial_data in group_fly.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)

    # Set axis limits (adjust as needed)
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    
    # Make axes equal
    ax.set_aspect('equal', adjustable='box')

    # Title and labels
    ax.set_title(f"Trajectories for ConfigFile: {config_of_interest}\nFlyID: {fly_id}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)

    # -----------------------------------------------------------
    # If it's a multi-goal config, calculate ratio left/right
    # -----------------------------------------------------------
    if is_multi_goal:
        # Filter the trials for this FlyID in results_df
        fly_results = results_df.loc[results_df['UniqueTrialID'].isin(group_fly['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        left_count = (fly_results['FirstReachedGoal'] == 'left').sum()
        right_count = (fly_results['FirstReachedGoal'] == 'right').sum()
        
        # Compute ratio (e.g., left/total and right/total)
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        # Add text box with ratio information
        ax.text(
            0.05, 0.95, ratio_text, 
            transform=ax.transAxes, va='top', 
            bbox=dict(boxstyle="round", fc="w", ec="0.5")
        )
    
    # -----------------------------------------------------------
    # If it's a single (center) goal config
    # -----------------------------------------------------------
    else:
        # Filter the trials for this FlyID in results_df
        fly_results = results_df.loc[results_df['UniqueTrialID'].isin(group_fly['UniqueTrialID'].unique())]
        
        # Count how many reached 'center'
        center_count = (fly_results['FirstReachedGoal'] == 'center').sum()
        
        ax.text(
            0.05, 0.95, f"Center goal only: {center_count}", 
            transform=ax.transAxes, va='top',
            bbox=dict(boxstyle="round", fc="w", ec="0.5")
        )
    
    # -----------------------------------------------------------
    # Save and show
    # -----------------------------------------------------------
    base_name = os.path.splitext(config_of_interest)[0]
    filename = f"trajectories_{base_name}_FlyID_{fly_id}.png"
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=300)

    plt.tight_layout()
    plt.show()

In [ ]:
# Group by FlyID and count how many unique UniqueTrialID values each FlyID has
counts_per_fly = df_cut.groupby('FlyID')['UniqueTrialID'].nunique()

# View this as a histogram of how many trials each FlyID had
plt.hist(counts_per_fly)
plt.xlabel("Number of unique trials per FlyID")
plt.ylabel("Frequency")
plt.title("Histogram of unique trials per FlyID")
plt.show()

In [ ]:
counts_per_fly = df_cut.groupby('FlyID')['UniqueTrialID'].nunique()
max_fly_id = counts_per_fly.idxmax()
max_trials = counts_per_fly.max()

print(f"The fly with the most unique trials is {max_fly_id}, "
      f"with {max_trials} unique trials.")


In [ ]:
counts_per_fly = df_cut.groupby("FlyID")["UniqueTrialID"].nunique()
flies_more_than_60 = counts_per_fly[counts_per_fly > 60]
print(flies_more_than_60)


In [ ]:
# Identify which ConfigFiles have multiple goals or triple goals
all_configs = df_cut['ConfigFile'].unique()
configs_2_goals = [
    "BinaryChoice_constantSize_GreenCylinder_GreenCylinder.json",
    "BinaryChoice11_constantSize_BlackCylinder_BlackCylinder.json",
    "BinaryChoice_constantSize_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_180deg_GreenCylinder_BlueCylinder.json",
    "BinaryChoice_constantSize_180deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_BlueCylinder_GreenCylinder.json",
    "BinaryChoice_constantSize_90deg_GreenCylinder_BlueCylinder.json"
]
# Merge results_df to get the goal reached information into df_cut (if needed)
df_joined = pd.merge(
    df_cut, 
    results_df[['UniqueTrialID', 'FirstReachedGoal']], 
    on='UniqueTrialID', 
    how='left'
)
df_joined = df_joined[df_joined['FlyID']=='307']
output_dir = '/Users/apaula/Downloads'

# Convert Current Time to just the date
df_joined["Day"] = df_joined["Current Time"].dt.date

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):

    # Extract the unique days for this config and map them to colors
    unique_days = group['Day'].unique()
    # Use a discrete set of colors from a colormap (e.g., rainbow)
    cmap = plt.cm.rainbow(np.linspace(0, 1, len(unique_days)))
    day_to_color = dict(zip(unique_days, cmap))

    # --- 1. Plot all trajectories for this config ---
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for trial_id, trial_data in group.groupby('UniqueTrialID'):

        # Identify the day for this trial (assuming each trial occurs in a single day)
        day_for_trial = trial_data['Day'].iloc[0]
        # Lookup color
        color = day_to_color[day_for_trial]

        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], color = color, alpha=0.3)
    
    # Basic plot settings
    ax.set_ylim(-10, 70)
    ax.set_xlim(-40, 40)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Trajectories for ConfigFile: {config}")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(True)
    
    # Create a legend showing which color corresponds to which day
    handles = []
    labels = []
    for d in unique_days:
        handles.append(plt.Line2D([], [], color=day_to_color[d], lw=2))
        labels.append(str(d))  # convert day to string for the legend
    ax.legend(handles, labels, title="Day", loc="best")
    
    # --- 2. Compute how many times each goal was reached for this config ---
    # Filter the relevant trials from results_df
    config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]

    # 2a. Single center-goal configs
    if config in center_only_configs:
        center_count = (config_results['FirstReachedGoal'] == 'center').sum()
        ratio_text = f"Center goal only: {center_count}"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2b. Triple-goal configs
    elif config in configs_3_goals:
        # Count how many reached each of the three possible goals
        left_count = (config_results['FirstReachedGoal'] == 'triple_left').sum()
        center_count = (config_results['FirstReachedGoal'] == 'triple_center').sum()
        right_count = (config_results['FirstReachedGoal'] == 'triple_right').sum()
        
        total = left_count + center_count + right_count
        if total > 0:
            left_ratio = left_count / total
            center_ratio = center_count / total
            right_ratio = right_count / total
            ratio_text = (
                f"L: {left_count} ({left_ratio:.2f}), "
                f"C: {center_count} ({center_ratio:.2f}), "
                f"R: {right_count} ({right_ratio:.2f})"
            )
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # 2c. Two-goal configs (assume everything else with multiple goals is two-goal)
    elif config in configs_2_goals:
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    # 2d. If you have other special cases or an 'else' fallback:
    else:
        # If you have some unclassified configs
        ratio_text = "Unclassified Config"
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top',
                bbox=dict(boxstyle="round", fc="w", ec="0.5"))

    # --- 3. Save and show the figure ---
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories4_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=300)
    
    plt.tight_layout()
    plt.show()

In [ ]:
df_joined ["Current Time"]